In [12]:
# coding: utf-8

from  __future__ import absolute_import
from __future__ import print_function
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model, load_model
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import layers
from keras import optimizers
import shutil
import random
from PIL import Image
import numpy as np
import logging
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import layers
import tensorflow_hub as hub
import shutil
import random
from PIL import Image
import pickle
import shutil
from tensorflow import debugging
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from pathlib import Path
from keras import applications
from keras import layers
from keras import losses
from keras import optimizers
from keras import metrics
from keras import Model
from keras.applications import resnet
from keras.callbacks import ModelCheckpoint

from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.impute import KNNImputer
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)


print(tf.__version__)

model_path = "./deep_ranking"

# batch_size = 96
# batch_size = 8
batch_size = 24
# batch_size = 3
# batch_size = 1

2.8.0


# Step 1: create lumo predictor

In [13]:

X = np.array(pd.read_csv("pretrain_features.csv.zip").iloc[:, 2:]).astype(np.float32)
Y = np.array(pd.read_csv("pretrain_labels.csv.zip").iloc[:, 1:]).astype(np.float32)

print(X.shape)
print(Y.shape)

# ridge = Ridge(alpha=0.5)
# ridge.fit(X, Y)

lumo_predictor = tf.keras.Sequential([
            layers.Dense(512, activation=tf.keras.activations.relu),
            layers.Dense(512, activation=tf.keras.activations.relu),
            layers.Dense(1, activation='linear')
        ])
lumo_predictor.compile(loss=losses.mean_squared_error, optimizer=tf.keras.optimizers.Adam(0.0001))
history = lumo_predictor.fit(X, Y, epochs=15)

lumo_predictor.save_weights('lumo_predictor.h5')

(50000, 1000)
(50000, 1)
Epoch 1/15
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1233
Epoch 2/15
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0108
Epoch 3/15
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0066
Epoch 4/15
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0048
Epoch 5/15
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0036
Epoch 6/15
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0029
Epoch 7/15
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0024
Epoch 8/15
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0020
Epoch 9/15
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0018
Epoch 10/15
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0015
Epoch 11/15
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0014
Epoch 12/15
1563/1563 [==============================] - 5s 

# Step 2: setup data to train homo predictor

In [14]:
X_train = np.array(pd.read_csv("train_features.csv.zip").iloc[:, 2:]).astype(np.float32)
Y_train = np.array(pd.read_csv("train_labels.csv.zip").iloc[:, 1:]).astype(np.float32)

Y_gap = np.array(pd.read_csv("train_labels.csv.zip").iloc[:, 1:]).astype(np.float32)

Y_lumo = lumo_predictor.predict(X_train)

Y_homo = -(Y_gap - Y_lumo)



# Step 3: train homo predictor

In [17]:
homo_predictor = tf.keras.Sequential([
            layers.Dense(512, activation=tf.keras.activations.relu),
            layers.Dense(512, activation=tf.keras.activations.relu),
            layers.Dense(1, activation='linear')
        ])

#build the model
homo_predictor.predict(X_train)

#transfer learning
homo_predictor.load_weights("lumo_predictor.h5")

homo_predictor.compile(loss=losses.mean_squared_error, optimizer=tf.keras.optimizers.Adam(0.00001))
history = homo_predictor.fit(X_train, Y_homo, epochs=150)



Epoch 1/150
4/4 [==============================] - 0s 3ms/step - loss: 3.5027
Epoch 2/150
4/4 [==============================] - 0s 4ms/step - loss: 3.3324
Epoch 3/150
4/4 [==============================] - 0s 4ms/step - loss: 3.1715
Epoch 4/150
4/4 [==============================] - 0s 4ms/step - loss: 3.0141
Epoch 5/150
4/4 [==============================] - 0s 3ms/step - loss: 2.8614
Epoch 6/150
4/4 [==============================] - 0s 3ms/step - loss: 2.7137
Epoch 7/150
4/4 [==============================] - 0s 4ms/step - loss: 2.5692
Epoch 8/150
4/4 [==============================] - 0s 4ms/step - loss: 2.4300
Epoch 9/150
4/4 [==============================] - 0s 3ms/step - loss: 2.2951
Epoch 10/150
4/4 [==============================] - 0s 4ms/step - loss: 2.1655
Epoch 11/150
4/4 [==============================] - 0s 4ms/step - loss: 2.0411
Epoch 12/150
4/4 [==============================] - 0s 4ms/step - loss: 1.9195
Epoch 13/150
4/4 [==============================] - 0s 4ms/st

# Step 4: test set

In [18]:
X_test = np.array(pd.read_csv("test_features.csv.zip").iloc[:, 2:]).astype(np.float32)
sample = pd.read_csv("sample.csv")


homo = homo_predictor.predict(X_test)
lumo = lumo_predictor.predict(X_test)

sample["y"] = (lumo-homo)

sample.to_csv('submit.csv', index=False, float_format='%.4f')